<a href="https://colab.research.google.com/github/alexabruck/speculative-datasets/blob/master/Forests_same_latitude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Images of forests on the same latitude as Leipzig

In [2]:
!pip install SPARQLWrapper

     |████████████████████████████████| 407 kB 11.6 MB/s 
     |████████████████████████████████| 45 kB 2.7 MB/s 


In [8]:
from SPARQLWrapper import SPARQLWrapper, JSON

ENDPOINT_URL = "https://query.wikidata.org/sparql"

def get(query):
    user_agent = "Speculative Datasets (https://github.com/alexabruck/speculative-datasets)"
    # TODO adjust user agent; see https://w.wiki/CX6
    sparql = SPARQLWrapper(ENDPOINT_URL, agent=user_agent)
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

query = """
SELECT ?item ?itemLabel ?locLatRef ?locLat ?locLongRef ?locLong ?pic
WHERE 
{ 
  ?item wdt:P31 wd:Q4421. #item is a forest
  
  ?item p:P625 ?locStatement.
  ?locStatement psv:P625 ?loc.
  ?loc wikibase:geoLongitude ?locLong.
  ?loc wikibase:geoLatitude ?locLat.
  
  wd:Q2079 p:P625 ?locRefStatement. #Ref = City of Leipzig, Germany
  ?locRefStatement psv:P625 ?locRef.
  ?locRef wikibase:geoLongitude ?locLongRef.
  ?locRef wikibase:geoLatitude ?locLatRef.
  
  ?item wdt:P18 ?image_. #has pic
  BIND(REPLACE(wikibase:decodeUri(STR(?image_)), "http://commons.wikimedia.org/wiki/Special:FilePath/", "") AS ?imageFileName_)
  BIND(REPLACE(?imageFileName_, " ", "_") AS ?imageFileNameSafe_)
  BIND(MD5(?imageFileNameSafe_) AS ?imageFileNameHash_)
  BIND(CONCAT("https://upload.wikimedia.org/wikipedia/commons/thumb/", SUBSTR(?imageFileNameHash_, 1 , 1 ), "/", SUBSTR(?imageFileNameHash_, 1 , 2 ), "/", ?imageFileNameSafe_, "/600px-", ?imageFileNameSafe_) AS ?pic)
  
  FILTER(?locLat < ?locLatRef + 0.5 && ?locLat > ?locLatRef - 0.5 )
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en". } # Helps get the label in your language, if not, then en language
}"""


response = get(query)
items = response["results"]["bindings"]
pics = [item['pic']['value'] for item in items]
print(len(pics))

144


## Download the pics

In [9]:
!rm -rd pics

In [ ]:
!mkdir pics

for pic in pics:
  !wget $pic -P pics

In [ ]:
!zip -r results.zip pics
from google.colab import files
files.download("results.zip")